# 🧠 Neural Storyteller – Image Captioning with Seq2Seq

**Objective:** Build a multimodal deep learning model that generates natural language descriptions for images using a Sequence-to-Sequence (Seq2Seq) architecture.

**Architecture:**
- **Encoder:** Pre-trained ResNet50 → Linear projection (2048 → 512)
- **Decoder:** LSTM with word embeddings, outputs vocabulary distribution

**Dataset:** Flickr30k (31,783 images with 5 captions each)

**Deployment:** All artifacts saved to `/content/deploy/` for Gradio Hugging Face Space

## 📦 Cell 1: Install Dependencies & Setup

In [ ]:
!pip install -q torch torchvision nltk gradio Pillow matplotlib tqdm kagglehub numpy pandas

import os
import re
import json
import math
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torchvision import models, transforms

import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score as nltk_meteor

# ── Reproducibility ──
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# ── Device ──
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ── Paths (Colab /content/) ──
BASE_DIR       = "/content"
FEATURES_FILE  = os.path.join(BASE_DIR, "flickr30k_features.pkl")
VOCAB_FILE     = os.path.join(BASE_DIR, "vocab.pkl")
MODEL_FILE     = os.path.join(BASE_DIR, "best_model.pth")
DEPLOY_DIR     = os.path.join(BASE_DIR, "deploy")
os.makedirs(DEPLOY_DIR, exist_ok=True)
print("✅ All dependencies installed and paths configured.")

## 📥 Cell 2: Download & Prepare Flickr30k Dataset

In [ ]:
import kagglehub

# Download Flickr30k from Kaggle  (authenticates via KAGGLE_USERNAME / KAGGLE_KEY)
dataset_path = kagglehub.dataset_download("adityajn105/flickr30k")
print(f"Dataset downloaded to: {dataset_path}")

# ── Locate images folder & captions file ──
IMAGE_DIR    = None
CAPTIONS_FILE = None

for root, dirs, files in os.walk(dataset_path):
    jpg_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg'))]
    if len(jpg_files) > 1000:
        IMAGE_DIR = root
    if 'captions.txt' in files:
        CAPTIONS_FILE = os.path.join(root, 'captions.txt')

# Fallback: also check for results.csv / captions.csv
if CAPTIONS_FILE is None:
    for root, dirs, files in os.walk(dataset_path):
        for f in files:
            if f.endswith('.csv') and 'caption' in f.lower():
                CAPTIONS_FILE = os.path.join(root, f)
                break
            if f == 'results.csv':
                CAPTIONS_FILE = os.path.join(root, f)
                break

assert IMAGE_DIR is not None,    "❌ Could not locate images folder!"
assert CAPTIONS_FILE is not None, "❌ Could not locate captions file!"

num_images = len([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg'))])
print(f"✅ Found {num_images} images at: {IMAGE_DIR}")
print(f"✅ Captions file at:  {CAPTIONS_FILE}")

## 🖼️ Part 1: Feature Extraction Pipeline with ResNet50

Extract a **2048-dimensional** feature vector from every image using a pre-trained ResNet50 (final classification layer removed). Features are cached to `flickr30k_features.pkl` so we only need to run this once.

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 1 – Feature Extraction Pipeline  (run ONCE, then skip)
# ════════════════════════════════════════════════════════════════

class FlickrImageDataset(Dataset):
    """Loads every .jpg from IMAGE_DIR, applies transform."""
    def __init__(self, img_dir, transform):
        self.img_dir = img_dir
        self.transform = transform
        self.img_names = sorted(
            [f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.jpeg'))]
        )

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        name = self.img_names[idx]
        img = Image.open(os.path.join(self.img_dir, name)).convert('RGB')
        return self.transform(img), name

# ── ResNet-50 feature extractor (remove final FC) ──
resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
resnet = nn.Sequential(*list(resnet.children())[:-1])          # output: (B, 2048, 1, 1)
resnet = nn.DataParallel(resnet).to(device)
resnet.eval()

img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

img_dataset = FlickrImageDataset(IMAGE_DIR, img_transform)
img_loader  = DataLoader(img_dataset, batch_size=128, num_workers=4, pin_memory=True)

features_dict = {}
with torch.no_grad():
    for imgs, names in tqdm(img_loader, desc="Extracting ResNet50 Features"):
        feats = resnet(imgs.to(device)).view(imgs.size(0), -1)   # (B, 2048)
        for i, name in enumerate(names):
            features_dict[name] = feats[i].cpu().numpy()

with open(FEATURES_FILE, 'wb') as f:
    pickle.dump(features_dict, f)

print(f"\n✅ Success! {len(features_dict)} images → 2048-d vectors saved to {FEATURES_FILE}")

## 📝 Part 2: Load Captions & Build Vocabulary

Load `captions.txt`, clean the text, and build a word-to-index vocabulary with special tokens `<pad>`, `<start>`, `<end>`, `<unk>`.

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 2 – Vocabulary & Text Pre-Processing
# ════════════════════════════════════════════════════════════════

# ── Load captions ──
captions_df = pd.read_csv(CAPTIONS_FILE, sep='|' if '|' in open(CAPTIONS_FILE).readline() else ',')
# Normalise column names
captions_df.columns = [c.strip().lower().replace(' ', '_') for c in captions_df.columns]
# Rename to standard names
rename_map = {}
for c in captions_df.columns:
    if 'image' in c:
        rename_map[c] = 'image'
    elif 'caption' in c:
        rename_map[c] = 'caption'
captions_df.rename(columns=rename_map, inplace=True)
captions_df.dropna(subset=['image', 'caption'], inplace=True)
captions_df['image'] = captions_df['image'].str.strip()
captions_df['caption'] = captions_df['caption'].str.strip()
print(f"Loaded {len(captions_df)} caption rows for {captions_df['image'].nunique()} images")

# ── Clean text ──
def clean_caption(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', '', text)                       # keep letters & spaces
    tokens = text.split()
    tokens = [t for t in tokens if len(t) > 1 or t in ('a', 'i')]  # drop single chars (except a/i)
    return ' '.join(tokens)

captions_df['caption_clean'] = captions_df['caption'].apply(clean_caption)
print("Sample cleaned captions:")
print(captions_df[['image', 'caption_clean']].head(10).to_string(index=False))

# ── Vocabulary class ──
class Vocabulary:
    PAD, START, END, UNK = '<pad>', '<start>', '<end>', '<unk>'

    def __init__(self, freq_threshold=5):
        self.freq_threshold = freq_threshold
        self.word2idx = {}
        self.idx2word = {}
        self._idx = 0
        # Add special tokens first
        for tok in [self.PAD, self.START, self.END, self.UNK]:
            self._add_word(tok)

    def _add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self._idx
            self.idx2word[self._idx] = word
            self._idx += 1

    def build(self, sentences):
        counter = Counter()
        for sent in sentences:
            counter.update(sent.split())
        for word, cnt in counter.items():
            if cnt >= self.freq_threshold:
                self._add_word(word)

    def numericalize(self, text):
        return [self.word2idx.get(w, self.word2idx[self.UNK]) for w in text.split()]

    def decode(self, indices):
        words = []
        for idx in indices:
            w = self.idx2word.get(idx, self.UNK)
            if w == self.END:
                break
            if w not in (self.PAD, self.START):
                words.append(w)
        return ' '.join(words)

    def __len__(self):
        return len(self.word2idx)

vocab = Vocabulary(freq_threshold=5)
vocab.build(captions_df['caption_clean'].tolist())
print(f"\n✅ Vocabulary size: {len(vocab)}")

# Save vocab
with open(VOCAB_FILE, 'wb') as f:
    pickle.dump(vocab, f)
print(f"Vocabulary saved to {VOCAB_FILE}")

## 🗂️ Part 3: Dataset Class & Train / Val / Test Split

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 3 – Caption Dataset & Data Loaders
# ════════════════════════════════════════════════════════════════

# Load cached features
with open(FEATURES_FILE, 'rb') as f:
    features_dict = pickle.load(f)
print(f"Loaded {len(features_dict)} image features")

# Keep only images that have both features AND captions
available_images = set(features_dict.keys()) & set(captions_df['image'].unique())
captions_df = captions_df[captions_df['image'].isin(available_images)].reset_index(drop=True)
print(f"Using {len(available_images)} images with {len(captions_df)} caption rows")

# ── Train / Val / Test split (80 / 10 / 10 by image) ──
all_images = sorted(available_images)
random.shuffle(all_images)
n = len(all_images)
train_imgs = set(all_images[:int(0.8 * n)])
val_imgs   = set(all_images[int(0.8 * n):int(0.9 * n)])
test_imgs  = set(all_images[int(0.9 * n):])
print(f"Split → Train: {len(train_imgs)}  Val: {len(val_imgs)}  Test: {len(test_imgs)}")

MAX_LEN = 40   # max caption length (tokens including <start>/<end>)

class CaptionDataset(Dataset):
    def __init__(self, df, features, vocab, max_len=MAX_LEN):
        self.data = df.reset_index(drop=True)
        self.features = features
        self.vocab = vocab
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_name = row['image']
        caption  = row['caption_clean']

        feat = torch.tensor(self.features[img_name], dtype=torch.float32)

        tokens = [self.vocab.word2idx[Vocabulary.START]] + \
                 self.vocab.numericalize(caption) + \
                 [self.vocab.word2idx[Vocabulary.END]]
        # Truncate
        tokens = tokens[:self.max_len]
        length = len(tokens)
        # Pad
        tokens += [self.vocab.word2idx[Vocabulary.PAD]] * (self.max_len - length)

        return feat, torch.tensor(tokens, dtype=torch.long), length

def collate_fn(batch):
    feats, caps, lengths = zip(*batch)
    feats   = torch.stack(feats)
    caps    = torch.stack(caps)
    lengths = torch.tensor(lengths, dtype=torch.long)
    # Sort by descending length for pack_padded_sequence
    sorted_idx = lengths.argsort(descending=True)
    return feats[sorted_idx], caps[sorted_idx], lengths[sorted_idx]

# ── Build DataFrames per split ──
train_df = captions_df[captions_df['image'].isin(train_imgs)]
val_df   = captions_df[captions_df['image'].isin(val_imgs)]
test_df  = captions_df[captions_df['image'].isin(test_imgs)]

BATCH_SIZE = 128

train_loader = DataLoader(CaptionDataset(train_df, features_dict, vocab),
                          batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate_fn, num_workers=2, pin_memory=True)
val_loader   = DataLoader(CaptionDataset(val_df, features_dict, vocab),
                          batch_size=BATCH_SIZE, shuffle=False,
                          collate_fn=collate_fn, num_workers=2, pin_memory=True)
test_loader  = DataLoader(CaptionDataset(test_df, features_dict, vocab),
                          batch_size=BATCH_SIZE, shuffle=False,
                          collate_fn=collate_fn, num_workers=2, pin_memory=True)

print(f"✅ DataLoaders ready  |  Train batches: {len(train_loader)}  "
      f"Val: {len(val_loader)}  Test: {len(test_loader)}")

## 🏗️ Part 4: Seq2Seq Architecture (Encoder + Decoder)

| Component | Details |
|-----------|---------|
| **Encoder** | Linear(2048 → 512) + BatchNorm + ReLU |
| **Decoder** | Embedding → LSTM → Linear(hidden → vocab_size) |
| **Teacher Forcing** | Used during training |

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 4 – Seq2Seq Model
# ════════════════════════════════════════════════════════════════

EMBED_SIZE  = 256
HIDDEN_SIZE = 512
NUM_LAYERS  = 1
DROPOUT     = 0.3

class Encoder(nn.Module):
    """Projects the 2048-d image feature to the LSTM hidden size."""
    def __init__(self, feature_dim=2048, hidden_size=HIDDEN_SIZE):
        super().__init__()
        self.fc   = nn.Linear(feature_dim, hidden_size)
        self.bn   = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(DROPOUT)

    def forward(self, features):
        # features: (B, 2048)
        x = self.drop(self.relu(self.bn(self.fc(features))))   # (B, hidden)
        return x


class Decoder(nn.Module):
    """LSTM decoder that produces a word at every time-step."""
    def __init__(self, vocab_size, embed_size=EMBED_SIZE,
                 hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super().__init__()
        self.embed  = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm   = nn.LSTM(embed_size, hidden_size,
                              num_layers=num_layers, batch_first=True, dropout=DROPOUT if num_layers > 1 else 0)
        self.fc_out = nn.Linear(hidden_size, vocab_size)
        self.drop   = nn.Dropout(DROPOUT)

    def forward(self, captions, hidden, cell):
        # captions: (B, T)   hidden/cell: (num_layers, B, hidden)
        embeds = self.drop(self.embed(captions))        # (B, T, embed)
        output, (hidden, cell) = self.lstm(embeds, (hidden, cell))  # (B, T, hidden)
        logits = self.fc_out(output)                    # (B, T, vocab_size)
        return logits, hidden, cell


class Seq2SeqCaptioner(nn.Module):
    def __init__(self, vocab_size, embed_size=EMBED_SIZE,
                 hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super().__init__()
        self.encoder    = Encoder(2048, hidden_size)
        self.decoder    = Decoder(vocab_size, embed_size, hidden_size, num_layers)
        self.num_layers = num_layers
        self.hidden_size = hidden_size

    def forward(self, features, captions):
        """
        features : (B, 2048)
        captions : (B, T)   – includes <start> ... <end> <pad>
        Returns  : logits (B, T-1, vocab_size)  (predict next word at each step)
        """
        enc_out = self.encoder(features)                         # (B, hidden)
        # Repeat for each LSTM layer
        h0 = enc_out.unsqueeze(0).repeat(self.num_layers, 1, 1) # (layers, B, hidden)
        c0 = torch.zeros_like(h0)

        # Teacher forcing: feed captions[:, :-1], predict captions[:, 1:]
        logits, _, _ = self.decoder(captions[:, :-1], h0, c0)
        return logits


# ── Instantiate ──
VOCAB_SIZE = len(vocab)
model = Seq2SeqCaptioner(VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable    = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(model)
print(f"\n✅ Total parameters : {total_params:,}")
print(f"   Trainable        : {trainable:,}")

## 🚀 Part 5: Training Loop with Validation

- **Loss:** CrossEntropyLoss (`ignore_index = <pad>`)
- **Optimizer:** Adam (lr = 1e-3)
- **Scheduler:** ReduceLROnPlateau
- **Early Stopping:** patience = 5

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 5 – Training & Validation
# ════════════════════════════════════════════════════════════════

PAD_IDX   = vocab.word2idx[Vocabulary.PAD]
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                        factor=0.5, patience=3)

NUM_EPOCHS     = 25
PATIENCE       = 5
best_val_loss  = float('inf')
patience_count = 0
train_losses   = []
val_losses     = []

for epoch in range(1, NUM_EPOCHS + 1):
    # ── TRAIN ──
    model.train()
    epoch_loss = 0.0
    for feats, caps, lengths in tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} [Train]", leave=False):
        feats, caps = feats.to(device), caps.to(device)
        logits = model(feats, caps)                       # (B, T-1, V)
        targets = caps[:, 1:]                              # (B, T-1)
        loss = criterion(logits.reshape(-1, VOCAB_SIZE), targets.reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()
        epoch_loss += loss.item()

    avg_train = epoch_loss / len(train_loader)
    train_losses.append(avg_train)

    # ── VALIDATE ──
    model.eval()
    epoch_val = 0.0
    with torch.no_grad():
        for feats, caps, lengths in tqdm(val_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} [Val]", leave=False):
            feats, caps = feats.to(device), caps.to(device)
            logits = model(feats, caps)
            targets = caps[:, 1:]
            loss = criterion(logits.reshape(-1, VOCAB_SIZE), targets.reshape(-1))
            epoch_val += loss.item()

    avg_val = epoch_val / len(val_loader)
    val_losses.append(avg_val)
    scheduler.step(avg_val)

    print(f"Epoch {epoch:02d}  |  Train Loss: {avg_train:.4f}  |  Val Loss: {avg_val:.4f}  |  LR: {optimizer.param_groups[0]['lr']:.6f}")

    # ── Checkpoint & Early Stop ──
    if avg_val < best_val_loss:
        best_val_loss = avg_val
        patience_count = 0
        torch.save(model.state_dict(), MODEL_FILE)
        print(f"   ✅ Best model saved (val loss {best_val_loss:.4f})")
    else:
        patience_count += 1
        if patience_count >= PATIENCE:
            print(f"   ⏹ Early stopping after {epoch} epochs")
            break

# Reload best weights
model.load_state_dict(torch.load(MODEL_FILE, map_location=device))
print(f"\n✅ Training complete. Best Val Loss: {best_val_loss:.4f}")

## 📉 Part 6: Loss Curves

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 6 – Plot Training & Validation Loss
# ════════════════════════════════════════════════════════════════

plt.figure(figsize=(10, 5))
epochs_range = range(1, len(train_losses) + 1)
plt.plot(epochs_range, train_losses, 'b-o', label='Training Loss')
plt.plot(epochs_range, val_losses,   'r-o', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Cross-Entropy Loss')
plt.title('Training & Validation Loss Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, 'loss_curve.png'), dpi=150)
plt.show()
print("✅ Loss curve saved to /content/loss_curve.png")

## 🔍 Part 7: Inference – Greedy Search & Beam Search

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 7 – Greedy Search & Beam Search
# ════════════════════════════════════════════════════════════════

@torch.no_grad()
def greedy_search(model, feature, vocab, max_len=MAX_LEN):
    """Generate a caption using greedy (argmax) decoding."""
    model.eval()
    feature = feature.unsqueeze(0).to(device)                    # (1, 2048)
    enc_out = model.encoder(feature)                              # (1, hidden)
    h = enc_out.unsqueeze(0).repeat(model.num_layers, 1, 1)
    c = torch.zeros_like(h)

    word_idx = vocab.word2idx[Vocabulary.START]
    words = []

    for _ in range(max_len):
        inp = torch.tensor([[word_idx]], device=device)           # (1, 1)
        logits, h, c = model.decoder(inp, h, c)                  # (1,1,V)
        word_idx = logits.argmax(dim=-1).item()
        word = vocab.idx2word[word_idx]
        if word == Vocabulary.END:
            break
        words.append(word)

    return ' '.join(words)


@torch.no_grad()
def beam_search(model, feature, vocab, beam_width=3, max_len=MAX_LEN):
    """Generate a caption using beam search."""
    model.eval()
    feature = feature.unsqueeze(0).to(device)
    enc_out = model.encoder(feature)
    h0 = enc_out.unsqueeze(0).repeat(model.num_layers, 1, 1)
    c0 = torch.zeros_like(h0)

    start_idx = vocab.word2idx[Vocabulary.START]
    end_idx   = vocab.word2idx[Vocabulary.END]

    # Each beam: (log_prob, token_list, h, c)
    beams = [(0.0, [start_idx], h0, c0)]
    completed = []

    for _ in range(max_len):
        new_beams = []
        for log_prob, seq, h, c in beams:
            inp = torch.tensor([[seq[-1]]], device=device)
            logits, h_new, c_new = model.decoder(inp, h, c)       # (1,1,V)
            log_probs = F.log_softmax(logits.squeeze(0).squeeze(0), dim=-1)  # (V,)
            topk_probs, topk_idx = log_probs.topk(beam_width)

            for k in range(beam_width):
                token = topk_idx[k].item()
                new_lp = log_prob + topk_probs[k].item()
                new_seq = seq + [token]
                if token == end_idx:
                    completed.append((new_lp / len(new_seq), new_seq))
                else:
                    new_beams.append((new_lp, new_seq, h_new, c_new))

        # Keep top beam_width beams
        new_beams.sort(key=lambda x: x[0], reverse=True)
        beams = new_beams[:beam_width]

        if len(beams) == 0:
            break

    if completed:
        completed.sort(key=lambda x: x[0], reverse=True)
        best_seq = completed[0][1]
    else:
        best_seq = beams[0][1]

    words = [vocab.idx2word[idx] for idx in best_seq
             if vocab.idx2word[idx] not in (Vocabulary.START, Vocabulary.END, Vocabulary.PAD)]
    return ' '.join(words)


# ── Quick test on one image ──
sample_img = list(test_imgs)[0]
sample_feat = torch.tensor(features_dict[sample_img], dtype=torch.float32)
print(f"Image: {sample_img}")
print(f"  Greedy : {greedy_search(model, sample_feat, vocab)}")
print(f"  Beam(3): {beam_search(model, sample_feat, vocab, beam_width=3)}")

## 🖼️ Part 8: Display 5 Random Test Images with Captions

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 8 – Show 5 Random Test Images with Captions
# ════════════════════════════════════════════════════════════════

sample_test_imgs = random.sample(sorted(test_imgs), 5)

fig, axes = plt.subplots(1, 5, figsize=(25, 6))
for ax, img_name in zip(axes, sample_test_imgs):
    img_path = os.path.join(IMAGE_DIR, img_name)
    img = Image.open(img_path).convert('RGB')
    ax.imshow(img)
    ax.axis('off')

    # Ground truth (first caption)
    gt = captions_df[captions_df['image'] == img_name]['caption_clean'].values[0]

    feat = torch.tensor(features_dict[img_name], dtype=torch.float32)
    greedy_cap = greedy_search(model, feat, vocab)
    beam_cap   = beam_search(model, feat, vocab, beam_width=3)

    ax.set_title(
        f"GT: {gt[:60]}...\n"
        f"Greedy: {greedy_cap}\n"
        f"Beam: {beam_cap}",
        fontsize=8, wrap=True
    )

plt.suptitle("Test Set – Ground Truth vs Generated Captions", fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, 'sample_captions.png'), dpi=150, bbox_inches='tight')
plt.show()

## 📊 Part 9: Quantitative Evaluation (BLEU-4, Precision, Recall, F1, METEOR)

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 9 – Quantitative Evaluation
# ════════════════════════════════════════════════════════════════

from nltk.tokenize import word_tokenize

# Group ground-truth captions per image for test set
test_gt = {}
for _, row in test_df.iterrows():
    img = row['image']
    cap = row['caption_clean']
    test_gt.setdefault(img, []).append(cap.split())

# Generate one prediction per test image
references = []
hypotheses = []
meteor_scores = []

print("Generating captions for test set...")
for img_name in tqdm(sorted(test_gt.keys())):
    feat = torch.tensor(features_dict[img_name], dtype=torch.float32)
    pred = beam_search(model, feat, vocab, beam_width=3)
    pred_tokens = pred.split()

    refs = test_gt[img_name]   # list of reference token-lists
    references.append(refs)
    hypotheses.append(pred_tokens)

    # METEOR (per-sentence, then averaged later)
    m = nltk_meteor(
        [word_tokenize(' '.join(r)) for r in refs],
        word_tokenize(pred)
    )
    meteor_scores.append(m)

# ── BLEU-4 ──
smoother = SmoothingFunction().method1
bleu4 = corpus_bleu(references, hypotheses,
                    weights=(0.25, 0.25, 0.25, 0.25),
                    smoothing_function=smoother)

# ── Token-level Precision / Recall / F1 ──
total_tp, total_fp, total_fn = 0, 0, 0
for refs, hyp in zip(references, hypotheses):
    ref_tokens = set()
    for r in refs:
        ref_tokens.update(r)
    hyp_set = set(hyp)
    tp = len(hyp_set & ref_tokens)
    fp = len(hyp_set - ref_tokens)
    fn = len(ref_tokens - hyp_set)
    total_tp += tp
    total_fp += fp
    total_fn += fn

precision = total_tp / (total_tp + total_fp + 1e-8)
recall    = total_tp / (total_tp + total_fn + 1e-8)
f1        = 2 * precision * recall / (precision + recall + 1e-8)
avg_meteor = np.mean(meteor_scores)

# ── Print Results ──
print("\n" + "=" * 50)
print("     QUANTITATIVE EVALUATION RESULTS")
print("=" * 50)
print(f"  BLEU-4     : {bleu4:.4f}")
print(f"  Precision  : {precision:.4f}")
print(f"  Recall     : {recall:.4f}")
print(f"  F1-Score   : {f1:.4f}")
print(f"  METEOR     : {avg_meteor:.4f}")
print("=" * 50)

eval_results = {
    "BLEU-4": round(bleu4, 4),
    "Precision": round(precision, 4),
    "Recall": round(recall, 4),
    "F1": round(f1, 4),
    "METEOR": round(avg_meteor, 4),
}
print(eval_results)

## 💾 Part 10: Save Model Artifacts to `/content/deploy/`

All files needed for the Gradio Hugging Face Space will be saved here.

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 10 – Save Deployment Artifacts to /content/deploy/
# ════════════════════════════════════════════════════════════════
import shutil

os.makedirs(DEPLOY_DIR, exist_ok=True)

# 1) Model weights
deploy_model_path = os.path.join(DEPLOY_DIR, "best_model.pth")
shutil.copy(MODEL_FILE, deploy_model_path)

# 2) Vocabulary
deploy_vocab_path = os.path.join(DEPLOY_DIR, "vocab.pkl")
shutil.copy(VOCAB_FILE, deploy_vocab_path)

# 3) Config JSON (hyper-parameters needed at inference)
config = {
    "embed_size":   EMBED_SIZE,
    "hidden_size":  HIDDEN_SIZE,
    "num_layers":   NUM_LAYERS,
    "vocab_size":   VOCAB_SIZE,
    "max_len":      MAX_LEN,
    "dropout":      DROPOUT,
    "beam_width":   3,
}
config_path = os.path.join(DEPLOY_DIR, "config.json")
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

# Verify
print("✅ Deployment artifacts saved to /content/deploy/")
for fname in os.listdir(DEPLOY_DIR):
    fpath = os.path.join(DEPLOY_DIR, fname)
    size_kb = os.path.getsize(fpath) / 1024
    print(f"   {fname:30s}  {size_kb:>8.1f} KB")

## 🌐 Part 11: Generate Gradio `app.py` for Hugging Face Space

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 11 – Write app.py for Gradio / HuggingFace Space
# ════════════════════════════════════════════════════════════════

app_code = r'''#!/usr/bin/env python3
"""Neural Storyteller – Gradio App for Hugging Face Spaces."""

import os, json, pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import gradio as gr

# ── Device ──
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ── Load config ──
with open("config.json", "r") as f:
    cfg = json.load(f)

EMBED_SIZE  = cfg["embed_size"]
HIDDEN_SIZE = cfg["hidden_size"]
NUM_LAYERS  = cfg["num_layers"]
VOCAB_SIZE  = cfg["vocab_size"]
MAX_LEN     = cfg["max_len"]
DROPOUT     = cfg["dropout"]
BEAM_WIDTH  = cfg["beam_width"]

# ── Load vocabulary ──
with open("vocab.pkl", "rb") as f:
    vocab = pickle.load(f)

# ── Model Definitions (must match training code) ──
class Encoder(nn.Module):
    def __init__(self, feature_dim=2048, hidden_size=HIDDEN_SIZE):
        super().__init__()
        self.fc   = nn.Linear(feature_dim, hidden_size)
        self.bn   = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(DROPOUT)

    def forward(self, features):
        return self.drop(self.relu(self.bn(self.fc(features))))


class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size=EMBED_SIZE,
                 hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super().__init__()
        self.embed  = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm   = nn.LSTM(embed_size, hidden_size,
                              num_layers=num_layers, batch_first=True,
                              dropout=DROPOUT if num_layers > 1 else 0)
        self.fc_out = nn.Linear(hidden_size, vocab_size)
        self.drop   = nn.Dropout(DROPOUT)

    def forward(self, captions, hidden, cell):
        embeds = self.drop(self.embed(captions))
        output, (hidden, cell) = self.lstm(embeds, (hidden, cell))
        logits = self.fc_out(output)
        return logits, hidden, cell


class Seq2SeqCaptioner(nn.Module):
    def __init__(self, vocab_size, embed_size=EMBED_SIZE,
                 hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS):
        super().__init__()
        self.encoder    = Encoder(2048, hidden_size)
        self.decoder    = Decoder(vocab_size, embed_size, hidden_size, num_layers)
        self.num_layers = num_layers
        self.hidden_size = hidden_size

    def forward(self, features, captions):
        enc_out = self.encoder(features)
        h0 = enc_out.unsqueeze(0).repeat(self.num_layers, 1, 1)
        c0 = torch.zeros_like(h0)
        logits, _, _ = self.decoder(captions[:, :-1], h0, c0)
        return logits


# ── Load trained weights ──
caption_model = Seq2SeqCaptioner(VOCAB_SIZE, EMBED_SIZE, HIDDEN_SIZE, NUM_LAYERS).to(device)
caption_model.load_state_dict(torch.load("best_model.pth", map_location=device))
caption_model.eval()

# ── ResNet50 feature extractor ──
resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet = resnet.to(device)
resnet.eval()

img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


# ── Beam Search ──
@torch.no_grad()
def beam_search_inference(feature, beam_width=BEAM_WIDTH):
    feature = feature.unsqueeze(0).to(device)
    enc_out = caption_model.encoder(feature)
    h0 = enc_out.unsqueeze(0).repeat(caption_model.num_layers, 1, 1)
    c0 = torch.zeros_like(h0)

    start_idx = vocab.word2idx[vocab.START]
    end_idx   = vocab.word2idx[vocab.END]

    beams = [(0.0, [start_idx], h0, c0)]
    completed = []

    for _ in range(MAX_LEN):
        new_beams = []
        for log_prob, seq, h, c in beams:
            inp = torch.tensor([[seq[-1]]], device=device)
            logits, h_new, c_new = caption_model.decoder(inp, h, c)
            log_probs = F.log_softmax(logits.squeeze(0).squeeze(0), dim=-1)
            topk_probs, topk_idx = log_probs.topk(beam_width)

            for k in range(beam_width):
                token = topk_idx[k].item()
                new_lp = log_prob + topk_probs[k].item()
                new_seq = seq + [token]
                if token == end_idx:
                    completed.append((new_lp / len(new_seq), new_seq))
                else:
                    new_beams.append((new_lp, new_seq, h_new, c_new))

        new_beams.sort(key=lambda x: x[0], reverse=True)
        beams = new_beams[:beam_width]
        if not beams:
            break

    if completed:
        completed.sort(key=lambda x: x[0], reverse=True)
        best_seq = completed[0][1]
    else:
        best_seq = beams[0][1] if beams else [start_idx]

    words = [vocab.idx2word[idx] for idx in best_seq
             if vocab.idx2word[idx] not in (vocab.START, vocab.END, vocab.PAD)]
    return " ".join(words)


# ── Prediction function for Gradio ──
def predict(image):
    """Take a PIL image → return generated caption string."""
    if image is None:
        return "Please upload an image."
    image = image.convert("RGB")
    img_tensor = img_transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        feature = resnet(img_tensor).view(1, -1).squeeze(0)

    caption = beam_search_inference(feature)
    return caption


# ── Gradio Interface ──
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload an Image"),
    outputs=gr.Textbox(label="Generated Caption"),
    title="🧠 Neural Storyteller – Image Captioning",
    description=(
        "Upload any image and this Seq2Seq model (ResNet50 encoder + LSTM decoder) "
        "trained on Flickr30k will generate a natural language caption using beam search."
    ),
    allow_flagging="never",
)

if __name__ == "__main__":
    demo.launch()
'''

app_path = os.path.join(DEPLOY_DIR, "app.py")
with open(app_path, 'w') as f:
    f.write(app_code)

print(f"✅ app.py written to {app_path}")
print(f"   Size: {os.path.getsize(app_path) / 1024:.1f} KB")

## 📄 Part 12: Generate `requirements.txt` & `README.md` for HF Space

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 12 – requirements.txt & README.md
# ════════════════════════════════════════════════════════════════

# ── requirements.txt ──
requirements = """torch
torchvision
gradio
Pillow
numpy
"""
req_path = os.path.join(DEPLOY_DIR, "requirements.txt")
with open(req_path, 'w') as f:
    f.write(requirements.strip() + '\n')
print(f"✅ requirements.txt → {req_path}")

# ── README.md (HuggingFace Space metadata) ──
readme = """---
title: Neural Storyteller Image Captioning
emoji: 🧠
colorFrom: blue
colorTo: purple
sdk: gradio
sdk_version: "4.44.0"
app_file: app.py
pinned: false
---

# 🧠 Neural Storyteller – Image Captioning with Seq2Seq

Upload any image and this model will generate a natural language caption!

## Architecture
- **Encoder:** Pre-trained ResNet50 (2048-d features) → Linear projection (512-d)
- **Decoder:** LSTM with learned word embeddings
- **Inference:** Beam Search (width = 3)

## Dataset
Trained on **Flickr30k** (31,783 images, 5 captions each).

## Metrics
| Metric | Score |
|--------|-------|
| BLEU-4 | See notebook |
| METEOR | See notebook |
| Token F1 | See notebook |

## Files
- `app.py` – Gradio application
- `best_model.pth` – Trained Seq2Seq weights
- `vocab.pkl` – Vocabulary (word ↔ index)
- `config.json` – Model hyper-parameters
"""
readme_path = os.path.join(DEPLOY_DIR, "README.md")
with open(readme_path, 'w') as f:
    f.write(readme)
print(f"✅ README.md → {readme_path}")

# ── Summary ──
print("\n" + "=" * 60)
print("  📦  /content/deploy/  –  READY FOR HUGGING FACE SPACE")
print("=" * 60)
for fname in sorted(os.listdir(DEPLOY_DIR)):
    fpath = os.path.join(DEPLOY_DIR, fname)
    size_kb = os.path.getsize(fpath) / 1024
    print(f"   {fname:30s}  {size_kb:>10.1f} KB")
print("=" * 60)
print("\n📋 Upload instructions:")
print("   1. Go to https://huggingface.co/new-space")
print("   2. Choose SDK → Gradio")
print("   3. Upload ALL files from /content/deploy/")
print("   4. Or use git:")
print("      git clone https://huggingface.co/spaces/YOUR_USER/YOUR_SPACE")
print("      cp /content/deploy/* YOUR_SPACE/")
print("      cd YOUR_SPACE && git add . && git commit -m 'init' && git push")

## 🧪 Part 13: Test Gradio App Locally in Colab

Run the cell below to launch the Gradio interface with a **public shareable link**.

In [ ]:
# ════════════════════════════════════════════════════════════════
# Part 13 – Launch Gradio App in Colab for Live Testing
# ════════════════════════════════════════════════════════════════

import sys
sys.path.insert(0, DEPLOY_DIR)

# Quick programmatic test on a sample test image
sample_img_name = random.choice(sorted(test_imgs))
sample_image = Image.open(os.path.join(IMAGE_DIR, sample_img_name)).convert('RGB')
print(f"Testing on: {sample_img_name}")

# Extract feature and generate caption using existing model (already loaded)
test_tensor = img_transform(sample_image).unsqueeze(0).to(device)
with torch.no_grad():
    # Use the resnet that's already loaded from Part 1 (remove DataParallel wrapper)
    resnet_single = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    resnet_single = nn.Sequential(*list(resnet_single.children())[:-1]).to(device)
    resnet_single.eval()
    test_feat = resnet_single(test_tensor).view(1, -1).squeeze(0)

caption = beam_search(model, test_feat, vocab, beam_width=3)
gt_caps = captions_df[captions_df['image'] == sample_img_name]['caption_clean'].values
print(f"  Ground Truth : {gt_caps[0]}")
print(f"  Generated    : {caption}")

# ── Launch Gradio inline ──
print("\n🚀 Launching Gradio interface...")

# Define predict function using loaded model
def colab_predict(image):
    if image is None:
        return "Please upload an image."
    image = image.convert("RGB")
    t = img_transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        feat = resnet_single(t).view(1, -1).squeeze(0)
    return beam_search(model, feat, vocab, beam_width=3)

demo = gr.Interface(
    fn=colab_predict,
    inputs=gr.Image(type="pil", label="Upload an Image"),
    outputs=gr.Textbox(label="Generated Caption"),
    title="🧠 Neural Storyteller – Image Captioning",
    description=(
        "Upload any image and this Seq2Seq model (ResNet50 encoder + LSTM decoder) "
        "trained on Flickr30k will generate a natural language caption using beam search."
    ),
    allow_flagging="never",
)

demo.launch(share=True, debug=False)